In [55]:
import sage.all
def better_bn(n):
    from itertools import product, permutations
    from sage.matrix.constructor import matrix

    signs = list(product([-1, 1], repeat=n))
    
    perms = list(permutations(range(n)))

    matrices = [matrix(n, n, [s[p[i]] if j == p[i] else 0 for i in range(n) for j in range(n)]) for s in signs for p in perms]

    return matrices

G = better_bn(3)
G
matrix = MatrixGroup(G)



table = matrix.character_table()
table
#table

# for elt in matrix.conjugacy_classes():
#     rep = elt.an_element()
#     conj = ConjugacyClass(matrix, rep)
#     print(conj.list())
#     print()


#below code used for large B_n, doesn't print properly with built-in commands
    
# for i in range(table.nrows()):
#     for j in range(table.ncols()):
#         print(table[i,j], end='  ')  # Adjust spacing as needed
#     print()  # To move to the next line

#table.str()


[ 1  1  1  1  1  1  1  1  1  1]
[ 1  1 -1  1 -1  1 -1  1 -1 -1]
[ 1  1 -1 -1  1 -1  1  1 -1  1]
[ 1  1  1 -1 -1 -1 -1  1  1 -1]
[ 2  2  0  0 -2  0  1 -1  0 -2]
[ 2  2  0  0  2  0 -1 -1  0  2]
[ 3 -1 -1  1 -1 -1  0  0  1  3]
[ 3 -1 -1 -1  1  1  0  0  1 -3]
[ 3 -1  1 -1 -1  1  0  0 -1  3]
[ 3 -1  1  1  1 -1  0  0 -1 -3]

In [56]:
def tuple_rep(matrix):
    n = matrix.nrows()
    left = []
    right = []
    visited = set()

    for i in range(n):
        if i in visited:
            continue
        
        cycle = 0
        parity = 1
        row = i

        while row != i or cycle == 0:
            visited.add(row)

            for j, elt in enumerate(matrix.row(row)):
                if elt != 0:
                    cycle += 1
                    parity *= elt
                    row = j
                    break

        if parity == 1:
            left.append(cycle)
        else:
            right.append(cycle)
    
    return [tuple(sorted(left, reverse=True)), tuple(sorted(right, reverse=True))]

tuples_list = []
for matrix in G:
    tuples_list.append(tuple_rep(matrix))

In [57]:
from sage.combinat.root_system.weyl_group import WeylGroup

n = 6 
W = WeylGroup(["B", n])
#char_table = W.character_table()
#char_table

In [58]:
def find_congruent_classes_v2(G, char_table, p_m):
    # Get the number of rows and columns in the character table
    num_rows, num_cols = char_table.dimensions()

    # Initialize a dictionary to store the congruent classes
    congruent_classes = {}
    
    conjugacy_classes = G.conjugacy_classes()


    # Iterate over the columns of the character table
    for i in range(num_cols):
#         if i not in added_columns:
            for j in range(i+1, num_cols):
                # Check if the two columns are congruent modulo p_m
                if all((ZZ(char_table[k,i]) - ZZ(char_table[k,j])) % p_m == 0 for k in range(num_rows)):
                    # If they are, add them to the dictionary
                    if str(conjugacy_classes[i]) not in congruent_classes:
                        congruent_classes[str(conjugacy_classes[i])] = [char_table.column(i)]
                    congruent_classes[str(conjugacy_classes[i])].append(char_table.column(j))
                else:
                    if str(conjugacy_classes[i]) not in congruent_classes:
                        congruent_classes[str(conjugacy_classes[i])] = [char_table.column(i)]
                        
    
    added_columns = set()
    temporary_dict = {}

    for key in congruent_classes:
        temporary_dict[key] = []
        
        for column in congruent_classes[key]: 
            if tuple(column) not in added_columns:
                temporary_dict[key].append(column)
                added_columns.add(tuple(column))

    clean_dict = {}

    for key in temporary_dict:
        if temporary_dict[key] != []:
            clean_dict[key] = temporary_dict[key]

    return clean_dict

    # Return the dictionary of congruent classes
    



#take character table and mod by prime power, find which columns are congruent. 

In [59]:
p = 3
m = 1
matrix = MatrixGroup(G)
result = find_congruent_classes_v2(matrix, table, p**m)

result


# for elt in matrix.conjugacy_classes():
#     rep = elt.an_element()
#     conj = ConjugacyClass(matrix, rep)
#     print(conj.list())
#     print()

{'Conjugacy class of [1 0 0]\n[0 1 0]\n[0 0 1] in Matrix group over Integer Ring with 48 generators': [(1, 1, 1, 1, 2, 2, 3, 3, 3, 3),
  (1, 1, 1, 1, -1, -1, 0, 0, 0, 0)],
 'Conjugacy class of [-1  0  0]\n[ 0 -1  0]\n[ 0  0  1] in Matrix group over Integer Ring with 48 generators': [(1, 1, 1, 1, 2, 2, -1, -1, -1, -1)],
 'Conjugacy class of [-1  0  0]\n[ 0  0 -1]\n[ 0 -1  0] in Matrix group over Integer Ring with 48 generators': [(1, -1, -1, 1, 0, 0, -1, -1, 1, 1)],
 'Conjugacy class of [-1  0  0]\n[ 0  0 -1]\n[ 0  1  0] in Matrix group over Integer Ring with 48 generators': [(1, 1, -1, -1, 0, 0, 1, -1, -1, 1)],
 'Conjugacy class of [-1  0  0]\n[ 0  1  0]\n[ 0  0  1] in Matrix group over Integer Ring with 48 generators': [(1, -1, 1, -1, -2, 2, -1, 1, -1, 1)],
 'Conjugacy class of [ 0 -1  0]\n[-1  0  0]\n[ 0  0  1] in Matrix group over Integer Ring with 48 generators': [(1, 1, -1, -1, 0, 0, -1, 1, 1, -1)],
 'Conjugacy class of [ 0 -1  0]\n[ 0  0 -1]\n[-1  0  0] in Matrix group over Integ

In [60]:
# This function returns pairs (character table column, conjugacy class corresponding to that specific element/col.) 
def congruent_classes_and_columns(G, char_table):
    num_rows, num_cols = char_table.dimensions()

    pairs = list()
    
    conjugacy_classes = G.conjugacy_classes()


    for i in range(num_cols):
        pairs.append([char_table.column(i), conjugacy_classes[i]])
        
    return pairs

def pairs_congruent_id_mod_prime_power(G, char_table, p, m):
    pairs = congruent_classes_and_columns(G, char_table)
    full_conjugacy_classes = find_congruent_classes_v2(matrix, table, p**m)
    trivial_conj = full_conjugacy_classes['Conjugacy class of [1 0 0]\n[0 1 0]\n[0 0 1] in Matrix group over Integer Ring with 48 generators']
    
    merged_conjugacy_classes = []
    for column in trivial_conj:
        for i in range(len(pairs)):
            if tuple(column) == tuple(pairs[i][0]):

                merged_conjugacy_classes.append(pairs[i][1])
    
    congruent_matrices = []
    for conj_class in merged_conjugacy_classes:
        congruent_matrices.extend(conj_class.list())
    #matrix.conjugacy_classes()[0].list()
    #print(congruent_matrices)


    congruent_tuples = list()
    for mtx in congruent_matrices:
        congruent_tuples.append(tuple_rep(mtx.matrix()))

    return congruent_tuples

In [61]:
congruent_tuples = pairs_congruent_id_mod_prime_power(matrix, table, 2, 1)
congruent_tuples

[[(1, 1, 1), ()],
 [(1,), (1, 1)],
 [(1,), (1, 1)],
 [(1,), (1, 1)],
 [(2,), (1,)],
 [(2,), (1,)],
 [(2,), (1,)],
 [(2,), (1,)],
 [(2,), (1,)],
 [(2,), (1,)],
 [(), (2, 1)],
 [(), (2, 1)],
 [(), (2, 1)],
 [(), (2, 1)],
 [(), (2, 1)],
 [(), (2, 1)],
 [(1, 1), (1,)],
 [(1, 1), (1,)],
 [(1, 1), (1,)],
 [(2, 1), ()],
 [(2, 1), ()],
 [(2, 1), ()],
 [(2, 1), ()],
 [(2, 1), ()],
 [(2, 1), ()],
 [(1,), (2,)],
 [(1,), (2,)],
 [(1,), (2,)],
 [(1,), (2,)],
 [(1,), (2,)],
 [(1,), (2,)],
 [(), (1, 1, 1)]]